## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Constantine,DZ,36.3650,6.6147,36.81,87,0,0.00,clear sky
1,1,Morondava,MG,-20.2833,44.2833,78.64,89,23,8.14,few clouds
2,2,Mar Del Plata,AR,-38.0023,-57.5575,60.87,78,0,8.99,clear sky
3,3,Flinders,AU,-34.5833,150.8552,84.18,62,94,5.99,overcast clouds
4,4,Luwingu,ZM,-10.2621,29.9271,61.45,97,100,7.11,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 83


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Morondava,MG,-20.2833,44.2833,78.64,89,23,8.14,few clouds
5,5,Atuona,PF,-9.8000,-139.0333,77.31,80,22,13.73,light rain
8,8,Mataura,NZ,-46.1927,168.8643,70.90,39,100,5.77,overcast clouds
12,12,Hobart,AU,-42.8794,147.3294,74.01,60,75,10.36,broken clouds
15,15,Hilo,US,19.7297,-155.0900,74.93,76,0,8.05,clear sky
17,17,Kapaa,US,22.0752,-159.3190,73.72,65,0,18.41,clear sky
21,21,Saint-Philippe,RE,-21.3585,55.7679,80.19,80,31,11.05,scattered clouds
23,23,Bambanglipuro,ID,-7.9500,110.2833,71.60,66,89,6.40,overcast clouds
24,24,Esperance,AU,-33.8667,121.9000,76.84,57,68,13.27,broken clouds
29,29,The Valley,AI,18.2170,-63.0578,77.09,66,4,18.72,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Morondava,MG,78.64,few clouds,-20.2833,44.2833,
5,Atuona,PF,77.31,light rain,-9.8000,-139.0333,
8,Mataura,NZ,70.90,overcast clouds,-46.1927,168.8643,
12,Hobart,AU,74.01,broken clouds,-42.8794,147.3294,
15,Hilo,US,74.93,clear sky,19.7297,-155.0900,
17,Kapaa,US,73.72,clear sky,22.0752,-159.3190,
21,Saint-Philippe,RE,80.19,scattered clouds,-21.3585,55.7679,
23,Bambanglipuro,ID,71.60,overcast clouds,-7.9500,110.2833,
24,Esperance,AU,76.84,broken clouds,-33.8667,121.9000,
29,The Valley,AI,77.09,clear sky,18.2170,-63.0578,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
import numpy as np

In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Morondava,MG,78.64,few clouds,-20.2833,44.2833,La Case Bambou
5,Atuona,PF,77.31,light rain,-9.8000,-139.0333,Villa Enata
8,Mataura,NZ,70.90,overcast clouds,-46.1927,168.8643,Ellie's Villa
12,Hobart,AU,74.01,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
15,Hilo,US,74.93,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel


In [17]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))